This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv(os.getcwd()+'/.env')
API_KEY = os.getenv('NASDAQ_API_KEY')

# print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
dataset = "FSE/"
ticker = "AFX_X/"
data_type = "data.json"
# Set limit to 1 to only get the most recent row.
limit = "limit=1"

r = requests.get("https://data.nasdaq.com/api/v3/datasets/"+dataset+ticker+data_type+"?"+limit+"&"+API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json_file = r.json()
print(json_file["dataset_data"])

{'limit': 1, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2000-06-07', 'end_date': '2020-12-01', 'frequency': 'daily', 'data': [['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None]], 'collapse': None, 'order': None}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# Q1: Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017.
start = "start_date=2017-01-01"
end = "end_date=2017-12-31"
r = requests.get("https://data.nasdaq.com/api/v3/datasets/"+dataset+ticker+data_type+"?"+start+"&"+end+"&"+API_KEY)

In [6]:
# Q2: Convert the returned JSON object into a Python dictionary.
data = r.json()['dataset_data']
print(type(data))

<class 'dict'>


In [7]:
# Q3: Calculate what the highest and lowest opening prices were for the stock in this period.
date = [i[data['column_names'].index('Date')] for i in data['data']]
o = [i[data['column_names'].index('Open')] for i in data['data']]
o_clean = [i for i in o if i != None]

print(f"\
Highest Opening Price: {max(o_clean)} on {date[o.index(max(o_clean))]}\n\
Lowest Opening Price: {min(o_clean)} on {date[o.index(min(o_clean))]}")

Highest Opening Price: 53.11 on 2017-12-14
Lowest Opening Price: 34.0 on 2017-01-24


In [8]:
# Q4: What was the largest change in any one day (based on High and Low price)?
high = [i[data['column_names'].index('High')] for i in data['data']]
low = [i[data['column_names'].index('Low')] for i in data['data']]
change_1 = [h - l for h, l in zip(high, low)]

print(f"Highest change: {max(change_1):.3f} on {date[change_1.index(max(change_1))]}")

Highest change: 2.810 on 2017-05-11


In [9]:
# Q5: What was the largest change between any two days (based on Closing Price)?
close = [i[data['column_names'].index('Close')] for i in data['data']]
change_2 = [0]
for i in range(len(close)):
    if i+1 == len(close):
        break
    else: 
        change_2.append(close[i+1]-close[i])

print(f"Highest change: {max(change_2):.3f} on {date[change_2.index(max(change_2))]}")

Highest change: 2.560 on 2017-08-08


In [10]:
# Q6: What was the average daily trading volume during this year?
trade_vol = [i[data['column_names'].index('Traded Volume')] for i in data['data']]

print(f"Average Daily Trade Volume: {sum(trade_vol)/len(trade_vol):.3f}")

Average Daily Trade Volume: 89124.337


In [11]:
# Q7: What was the median trading volume during this year?
def median(l):
    """Returns the median of a list"""
    sorted_l = sorted(l)
    if len(sorted_l) % 2 == 1:
        med = sorted_l[int(len(sorted_l)//2)]
    else: 
        med = (sorted_l[len(sorted_l)/2] + sorted_1[(len(sorted_l)/2)+1]) / 2
    return med
        
print(f"Median Trade Volume: {median(trade_vol)}")

Median Trade Volume: 76286.0
